<a href="https://colab.research.google.com/github/LukeD77/Machine_Learning2/blob/master/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
''' For the structure:
DONE 1. get the data pipeline to flow into a file?
	a. Can we specify tweets with the word mask before we download?
DONE 2. Import that file into pycharm
DONE 3. Extract tweets into main dataframe
	a. Potentially have to filter out for tweets with just the word mask
DONE 4. Extract the url for the image of the profile picture and put into main data frame
5. Run connotative analysis and put the connotation in the dataframe
	a. check to see the unique values and throw out odd ones
6. Use a wget command or something like that to load all the images from the urls into a dataframe
7. Run that image list into the gender model and put the age and gender back into the main data frame
	a. check to see if images were succesfully identified and throw out ones that were nt
8. Split the dataframe or do kfold validation on random forest regression'''

' For the structure:\nDONE 1. get the data pipeline to flow into a file?\n\ta. Can we specify tweets with the word mask before we download?\nDONE 2. Import that file into pycharm\nDONE 3. Extract tweets into main dataframe\n\ta. Potentially have to filter out for tweets with just the word mask\nDONE 4. Extract the url for the image of the profile picture and put into main data frame\n5. Run connotative analysis and put the connotation in the dataframe\n\ta. check to see the unique values and throw out odd ones\n6. Use a wget command or something like that to load all the images from the urls into a dataframe\n7. Run that image list into the gender model and put the age and gender back into the main data frame\n\ta. check to see if images were succesfully identified and throw out ones that were nt\n8. Split the dataframe or do kfold validation on random forest regression'

In [2]:
## Problems ##
# the tweets i get are truncated and cut off so maybe find function that uses the url to go get the whole tweet
# Need to get profile_image_url to use facial recognition

In [5]:
import json
import nltk
from nltk import word_tokenize
import pandas as pd
import tweepy
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [6]:

### Retrieve data from Twitter
# Get authorizations
API_key = "m6ts9AIJlo6m3UBjA2ZKGhtIB"
API_secret_key = "DJgbXa1GSDS2wSPMQWlYmAylMRJUzjE4WGLgshQtOvAGng3NdF"
access_token = "1058432396482220032-ltlhoR2uAgogVZ3HrrfE5waU1SQUSD"
secret_access_token = "njNuN9e6JIDFrjuYbHUDeR400z3tCX12I2Vg0AusFWkD5"

# Set up API
auth = tweepy.OAuthHandler(API_key, API_secret_key)
auth.set_access_token(access_token, secret_access_token)
api = tweepy.API(auth)


In [8]:
# Search for tweets with the phrase "wear a mask" and put the tweet and profile image URL into a dataframe
tweet_list = []
image_list = []
for status in tweepy.Cursor(api.search,q='wear a mask',count=100, since=2020-3-1).items():
    tweet_list.append(status.text)
    image_list.append(status.user.profile_image_url)

# I could also do this again with the phrase "wearing a mask", "wore a mask" and such to expand dataset

len(tweet_list)
len(image_list)
main_df = pd.DataFrame()
main_df['tweets'] = tweet_list
main_df['images'] = image_list
main_df

TweepError: ignored

In [ ]:

### Using sentiment analysis model from tensorflow Text Classification with RNN

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

encoder = info.features['text'].encoder

for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)


def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [ ]:
# Run the model
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:

test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

# predict on a sample text without padding.

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

# predict on a sample text with padding

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

plot_graphs(history, 'accuracy')

plot_graphs(history, 'loss')

pred_text = main_df['tweets'][0]
pred_text

predictions = sample_predict(pred_text, pad=True)
print(predictions)

In [ ]:
sentiment_list = []
for tweet in main_df['tweets']:
  pred_text = tweet
  prediction = sample_predict(pred_text, pad = True)
  sentiment_list.append(prediction)

In [ ]:
for value in sentiment_list:
  if value < 0.5:
    value = 'Negative'
  else:
    value = 'Positive'

In [ ]:

########### Junk Yard ##############

#
# api = twitter.Api(consumer_key=API_key,
#                   consumer_secret=API_secret_key,
#                   access_token_key=access_token,
#                   access_token_secret=secret_access_token,
#                   tweet_mode='extended')
#
# data = api.GetSearch(term='mask',count=100, since=2020-3-1)
# data2 = api.GetSearch(term='masks', since=2020-3-1)
# data3 = api.GetStatus()
#
# #data = data.append(data2)
# main_df = pd.DataFrame()
#
# # extract tweet
# print(data)
# len(data)
# print(data2)
# test = data[0]
# test
# test[0]
#
# print(test.text)
#
# for status in api.Cursor(api.GetSearc)
#     n = n+1
#     print(n)
#
# # This does not work as a for loop because indexing data? changed to while loop and integer values and still didn't take
# l_tweets = []
# for x in data:
#     a = str(data[x])
#     b = a.split("text")
#     c = b[1].split('urls')
#     d = c[0].split()
#     e = d[1:-1]
#     f = ' '.join(e)
#     g = f[:-2]
#     h = g[1:]
#     tweet = h
#     l_tweets.append(tweet)
#
#
# # By hand
# a = str(data[2])
# b = a.split("text")
# c = b[1].split('urls')
# d = c[0].split()
# e = d[1:-1]
# f = ' '.join(e)
# g = f[:-2]
# h = g[1:]
# tweet = h
# l_tweets.append(tweet)
#
# main_df['tweets'] = l_tweets
# main_df
# pd.options.display.max_colwidth=300
# print(main_df.iloc[0],)
#
#
# a = str(data[3])
# b = a.split("text")
# c = b[1].split('urls')
# d = c[0].split()
# e = d[1:-1]
# f = ' '.join(e)
# g = f[:-2]
# h = g[1:]
# h
# print(a)
# data[3]
#
# a = str(data2[3])
# b = a.split("text")
# c = b[1].split('urls')
# d = c[0].split()
# e = d[1:-1]
# f = ' '.join(e)
# g = f[:-2]
# h = g[1:]
# h
# print(a)
# data[3]
#
# data_dct = dict()
